# Simulación sistema baño. 
### ANC. $P(Z):\text{baño}$. $W(z) = P(z)$ No adaptativo. Entrada: WGN. Audio deseado: Inyecto Musica. Me quedo con el error.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy import fft
import IPython.display as ipd
from scipy.io import wavfile
import os
from scipy import linalg

In [6]:
file_path = "auxiliary_database/fir_coefficients.txt"

fir_coefficients = np.loadtxt(file_path)

fs, speech_file = wavfile.read('auxiliary_database/musica_raw.wav')
speech_file = speech_file.astype(np.float32) / np.iinfo(speech_file.dtype).max
speech_file = speech_file[int(5*fs):]
z = signal.fftconvolve(speech_file, fir_coefficients, mode='full', axes=None) 
print('raw song:')
display(ipd.Audio(speech_file, rate=fs))
print('reverbered:')
display(ipd.Audio(z, rate=fs))

raw song:


reverbered:


# Construcción del sistema

In [32]:
# Esto es absolutumente trivial
def basic_ANC(x, sound):
    d = signal.fftconvolve(x, fir_coefficients, mode='full', axes=None)
    pre_d_hat = signal.fftconvolve(x, fir_coefficients, mode='full', axes=None)
    d_hat = pre_d_hat + sound[:len(pre_d_hat)]
    e = d - d_hat
    return e

In [61]:
x = 2 * np.random.normal(size=int(fs*3))
e = basic_ANC(x, speech_file)
print('Ultra ruidoso:')
display(ipd.Audio(x+speech_file[0:int(fs*3)], rate=fs))
print('Error:')
display(ipd.Audio(e, rate=fs))

Ultra ruidoso:


Error:


# Implementación de LMS, para que haga la identificación del sistema $P(z)$

In [ ]:
def get_model_output(x, fir_coefficients):
    '''
    Calcula la salida del modelo
    '''
    return signal.fftconvolve(x, fir_coefficients, mode='full', axes=None)

In [7]:
def calc_lms_montecarlo(K, w0, mu, model_system_coeffs, orden_filtro = 1):
    """
    Realiza una simulación de Monte-Carlo del algoritmo LMS.
    
    K: número de simulaciones de Monte-Carlo
    w0: valor inicial del filtro adaptativo
    mu: parámetro de paso
    N: número de iteraciones
    """
    M = len(model_system_coeffs)
    N = orden_filtro*10 if M < orden_filtro  else M*10
    # Simulación de Monte Carlo
    w_montecarlo = np.zeros((orden_filtro))
    J_montecarlo = np.zeros((N))
    
    for i in range(K):
        w = np.zeros(orden_filtro)
        J = np.zeros(N)
        w = w0
        if M < orden_filtro:
            x = np.random.normal(size = N + orden_filtro)
            
            d = signal.lfilter(b = model_system_coeffs, a = [1], x = x[orden_filtro - M:])[M:]
            for n in range(N):
                d_hat = np.dot(w, x[n + orden_filtro: n: -1])             # Ecuación de filtrado
                e = d[n] - d_hat
                J[n] = e * e
                w = w + mu * x[n + orden_filtro: n: -1] * e     # Ecuación LMS
                display(n)
        else:
            x = np.random.normal(size = N + M)
            d = signal.lfilter(b = model_system_coeffs, a = [1], x = x)[M:]
            for n in range(N):
                d_hat = np.dot(w, x[n + M: n + M - orden_filtro: -1])             # Ecuación de filtrado
                e = d[n] - d_hat
                J[n] = e * e
                w = w + mu * x[n + M: n + M - orden_filtro: -1] * e     # Ecuación LMS
                display(n)
        # Predicción LMS

        

            
        
        w_montecarlo += w
        J_montecarlo += J
        
    w_montecarlo /= K
    J_montecarlo /= K
    
    return (w_montecarlo, J_montecarlo)

In [8]:
w_order = 38400
w, J = calc_lms_montecarlo(1, np.zeros(w_order), 0.02, fir_coefficients, w_order)
# w1, J1 = calc_lms_montecarlo(100, np.zeros(4), 0.1, 4)

In [2]:
# plt.figure()

# plt.plot(J)
# plt.semilogy(J1)

display(w)
display(fir_coefficients[:10])

NameError: name 'w' is not defined